# Recolección data CoAID(Covid-19 heAlthcare mIsinformation Dataset)
La desinformación relacionada con COVID-19 también se crea y se propaga como la pólvora. Esta desinformación ha causado confusión entre las personas, trastornos en la sociedad e incluso consecuencias mortales en problemas de salud.

Este notebook nos permite obtener la data almacenada en el siguiente repo: https://github.com/cuilimeng/CoAID y almacenarla en una base de datos local.

Asimismo, solo los tweets son hidratados mediante la API de Twitter, para cual se desarrolló un algoritmo.


In [38]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))
from utils import get_connection_covid_coaid,get_df_sql
import pandas as pd
from IPython.display import clear_output
import math
import os
import os.path as osp
import argparse
from time import sleep

In [32]:
conn = get_connection_covid_coaid()
BASE_URL_DATASET = "https://raw.githubusercontent.com/cuilimeng/CoAID/master/"
DATA_DIR= '/home/reisson/Documents/platzi_interview/owid/CoAID/data'
os.mkdir(DATA_DIR)
PATH_FAKE_TWEETS = DATA_DIR+'/fake-tweets'
PATH_REAL_TWEETS = DATA_DIR+'/real-tweets'
CONSUMER_KEY='GV8cYa5GSjk27F2JZb6z8MrsJ'
CONSUMER_SECRET_KEY='mybd68vqhP6wSyvQOtxlYoc9aRne08FAImpchhL679W2vgNfsS'
ACCESS_TOKEN_KEY='1085983643640709121-qEMfxY6RSmPHHaEvDxIIsEXSjn2y5v'
ACCESS_TOKEN_SECRET_KEY='jZ02dDaFlPNC5gJI5PTuV9BeivolkPlPxcLK8LuAPDEME'
DATE_ARR = ['05-01-2020','07-01-2020','09-01-2020','11-01-2020']

In [ ]:
auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET_KEY)
auth.set_access_token(ACCESS_TOKEN_KEY,ACCESS_TOKEN_SECRET_KEY)
api = tweepy.API(auth, wait_on_rate_limit=True, 
                 retry_delay=60*3, 
                 retry_count=5,
                 retry_errors=set([401, 404, 500, 503]), 
                 wait_on_rate_limit_notify=True)

In [33]:
def prepare_tweet(tweet):
    try:        
        indices = [i for i, x in enumerate(tweet) if x == "'"]
        if indices == []:
            return tweet
        var = 0
        for a in indices:
            tweet = tweet[:a+var] + "'" + tweet[a+var:]
            var = var + 1
        return tweet
    except:
        return tweet

In [34]:
def call_sp_ins_class(tweet_id,tweet_class,line_count):
    cur = conn.cursor()
    try:
        call_sp = f"CALL public.ins_class_tweet('{tweet_id}','{tweet_class}');"
        cur.execute(call_sp)
        conn.commit()
        line_count=line_count+1
        print(f'Processed {line_count} lines.')
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        cur.close()
    return line_count

In [35]:
def is_retweet(entry):
    return 'retweeted_status' in entry.keys()

In [36]:
def get_source(entry):
    if '<' in entry["source"]:
        return entry["source"].split('>')[1].split('<')[0]
    else:
        return entry["source"]

In [ ]:
def call_sp_ins_class(tweet_id,tweet_class,line_count):
    cur = conn.cursor()
    try:
        call_sp = f"CALL public.ins_class_tweet('{tweet_id}','{tweet_class}');"
        cur.execute(call_sp)
        conn.commit()
        line_count=line_count+1
        print(f'Processed {line_count} lines.')
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        cur.close()
    return line_count

In [ ]:
def ins_class_tweet_from_csv(path_csv):
    line_count = 0
    class_tweet = path_csv.split('/')[-1].split('-')[0]
    with open(path_csv) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if row[1] != 'tweet_id':
                line_count = call_sp_ins_class(row[1],class_tweet,line_count)


In [37]:
def call_sp_ins_hydrated_tweet(data):
    CONN = psycopg2.connect(host=HOST,
                            database=DATABASE,
                            user=USER,
                            password=PASSWORD)
    call_sp=''
    
    tweet_prepared = prepare_tweet(data["text"])
    source_data = prepare_tweet(get_source(data))
    
    try:
        cur = conn.cursor()
        call_sp += "CALL public.insert_hydrated_tweet('"
        call_sp += str(data["id_str"]) + "',quote_literal('"
        call_sp += tweet_prepared + "'),NULL,'"
        call_sp += data["created_at"] + "',"
        call_sp += str(is_retweet(data)) + ",'"
        call_sp += source_data + "',"
        call_sp += str(data["favorite_count"]) + ","
        call_sp += str(data["retweet_count"]) + ")"
        cur.execute(call_sp)
        conn.commit()
    except(Exception,psycopg2.DatabaseError) as error:
        print(f'Se encontro el siguiente error: {error}')
    finally:
        cur.close()

In [ ]:
for i in DATE_ARR:
    print('read database '+ i)
    url_fake_news = BASE_URL_DATASET+i+"/"+"NewsFakeCOVID-19_tweets.csv?raw=true"
    path_fake_news = PATH_FAKE_TWEETS+'-'+i+'.csv'
    url_real_news = BASE_URL_DATASET+i+"/"+"NewsRealCOVID-19_tweets.csv?raw=true"
    path_real_news = PATH_REAL_TWEETS+'-'+i+'.csv'
    wget.download(url_fake_news, out=path_fake_news)
    wget.download(url_real_news, out=path_real_news)
    ins_class_tweet_from_csv(path_fake_news)
    ins_class_tweet_from_csv(path_real_news)
    clear_output()

In [ ]:
start = 0
end = 100
limit = len(ids)
print(f'Len ids:{len(ids)}')
i = int(math.ceil(float(limit) / 100))

output_file = DATA_DIR + '/json_data'

last_tweet = None

auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET_KEY)
auth.set_access_token(ACCESS_TOKEN_KEY,ACCESS_TOKEN_SECRET_KEY)
api = tweepy.API(auth, wait_on_rate_limit=True, 
                 retry_delay=60*3, 
                 retry_count=5,
                 retry_errors=set([401, 404, 500, 503]), 
                 wait_on_rate_limit_notify=True)

    
if api.verify_credentials() == False: 
    print('Credenciales no validas') 
    sys.exit()
else: 
    print("Credenciales validas") 

if osp.isfile(output_file) and osp.getsize(output_file) > 0:
    with open(output_file, 'rb') as f:
            #may be a large file, seeking without iterating
        f.seek(-2, os.SEEK_END)
        while f.read(1) != b'\n':
            f.seek(-2, os.SEEK_CUR)
        last_line = f.readline().decode()
    last_tweet = json.loads(last_line)
    print(ids.index((last_tweet['id']))) 
    start = ids.index(last_tweet['id'])
    end = start+100
    i = int(math.ceil(float(limit-start) / 100))

print('Coleccion de metadata')
print('Creando master json file')

try:
    with open(output_file, 'a') as outfile:
        for go in range(i):
            print('Obteniendo el rango {} - {}'.format(start, end))
            sleep(6)  # needed to prevent hitting API rate limit
            id_batch = ids[start:end]
            start += 100
            end += 100       
            backOffCounter = 1
            while True:
                try:
                    tweets = api.statuses_lookup(id_batch)
                    break
                except tweepy.TweepError as ex:
                    print('Se capturo TweepError exception:\n %s' % ex)
                    sleep(30*backOffCounter)  # sleep a bit to see if connection Error is resolved before retrying
                    backOffCounter += 1  # increase backoff
                    continue

            for tweet in tweets:
                call_sp_ins_hydrated_tweet(tweet._json)
                json.dump(tweet._json, outfile)
                outfile.write('\n')
except:
    print('exception: continuing to zip the file' + ex)